In [1]:
!pip install numpy pandas

In [1]:
import pandas as pd
import numpy as np
# from pygdf.dataframe import DataFrame
# from pygdf.index import GenericIndex, Index
# from numba import cuda, vectorize, jit
import json
import time
# import _thread as thread

In [2]:
data_set = pd.read_csv('./netflix-prize-data/combined_data_1.txt', header = None, 
                       names = ['cust_id', 'rating', 'date'])
data_set.head(5)

,cust_id,rating,date
0,1:,NaN,NaN
1,1488844,3.0,2005-09-06
2,822109,5.0,2005-05-13
3,885013,4.0,2005-10-19
4,30878,4.0,2005-12-26


In [3]:
print ("comecouu")
date_df = data_set.date.str.extract('(?P<year>\d{4})-(?P<month>\d{2})-(?P<day>\d{2}$)')
print ("terminou extract")
data_set.drop('date', axis=1, inplace=True)
print ("terminou drop date")
data_set = data_set.join(date_df)
print ("modelo final")
index = list(data_set.query("rating != rating").index)
print ("Filmes" , len(index))
filmes = list(map(lambda x: x[0], data_set.loc[index, "cust_id"].str.split(":")))
print ("Pegou os ids dos filmes")
data_set.loc[index, "cust_id"] = filmes
print ("salvando o arquivo")
data_set.to_csv("./combined_data_1.csv", index=False)
        

comecouu
terminou extract
terminou drop date
modelo final
Filmes 4499
Pegou os ids dos filmes
salvando o arquivo


In [14]:
data_set = pd.read_csv('./combined_data_1.csv')
data_set.head(5)

,cust_id,rating,year,month,day
0,1,NaN,NaN,NaN,NaN
1,1488844,3.0,2005.0,9.0,6.0
2,822109,5.0,2005.0,5.0,13.0
3,885013,4.0,2005.0,10.0,19.0
4,30878,4.0,2005.0,12.0,26.0


In [15]:
index = list(data_set.query("rating != rating").index)
print (len(index))

print ("Associando os filmes aos usuários que assistiram esse filme")
for i in range(0, len(index) - 1):
    data_set.loc[index[i]:index[i + 1], "filme"] = data_set.cust_id[index[i]]
    if (i % 5 == 0):
        progress = 100*(i/len(index))
        print ("[%s%s] %2.1f%%" % ('#'*int(progress), '_'*(100-int(progress)), progress), end="\r")
        
data_set.loc[index[-1]: len(data_set), "filme"] = data_set.cust_id[index[-1]]
data_set.drop(index, inplace=True)
print ("[%s] 100%%   " % ('#'*100))

print ("salvando o arquivo")
data_set.to_csv("./combined_data_1_filmes.csv", index=False)

4499
Associando os filmes aos usuários que assistiram esse filme
[####################################################################################################] 100%   
salvando o arquivo


In [13]:
data_set = pd.read_csv('./combined_data_1_filmes.csv')
data_set.head(5)

,cust_id,rating,year,month,day,filme
0,1488844,3.0,2005.0,9.0,6.0,1.0
1,822109,5.0,2005.0,5.0,13.0,1.0
2,885013,4.0,2005.0,10.0,19.0,1.0
3,30878,4.0,2005.0,12.0,26.0,1.0
4,823519,3.0,2004.0,5.0,3.0,1.0


In [14]:
start = time.time()
data_set_size = len(data_set)
users = {}
for i, avaliacao in data_set.iterrows():
    if (i % 100 == 0):
        progress = 100*(i/data_set_size)
        print ("[%s%s] %2.1f%%" % ('#'*int(progress), '_'*(100-int(progress)), progress), end="\r")
        
    filme = {
            "id": avaliacao["filme"],
            "avaliacao": avaliacao["rating"],
            "year": avaliacao["year"],
            "month": avaliacao["month"],
            "day": avaliacao["day"]
        }
    if avaliacao["cust_id"] in users:
        users[avaliacao["cust_id"]]["filmes"].append(filme)
    else:
        users[avaliacao["cust_id"]] = {
            "id": avaliacao["cust_id"],
            "filmes": []
        }
        users[avaliacao["cust_id"]]["filmes"].append(filme)
        
print ("[%s] 100%%   " % ('#'*100))

print ("Salvando no arquivo json")
with open('combined_data_1.json', 'w', encoding='utf8') as outfile:
    first = True
    users_size = len(users)
    i = 0
    for key, value in users.items():
        i += 1
        if (i % 10 == 0):
            progress = 100*(i/users_size)
            print ("[%s%s] %2.1f%%" % ('#'*int(progress), '_'*(100-int(progress)), progress), end="\r")
        
        avaliacoes = list(map(lambda x: x["avaliacao"], value["filmes"]))
        media = sum(avaliacoes)/len(avaliacoes)
        value["media"] = media
        value["user_similarity"] = []
        
        if (first):
            outfile.write("[\n" + json.dumps(value,
                                indent=4,
                                separators=(',', ': ')))
            first = False
        else:
            outfile.write(",\n" + json.dumps(value,
                                indent=4,
                                separators=(',', ': ')))
    outfile.write("\n]")

print ("[%s] 100%%   " % ('#'*100))
    
print (time.time() - start)

[####################################################################################################] 100%   
Salvando no arquivo json
[####################################################################################################] 100%   
6924.083045959473


In [ ]:
print ("Pegando os ids dos usuários")
# unique_users = df.cust_id.unique()
# len_unique_user = len(unique_users)

print ("Associando os usuários com os filmes que eles assistiram")
usuarios = []
for index, cust_id in enumerate(unique_users):
    if (index % 5 == 0):
        progress = 100*(index/len_unique_user)
        print ("[%s%s] %2.1f%%" % ('#'*int(progress), '_'*(100-int(progress)), progress), end="\r")
    lista_assistiu = df[df.cust_id == cust_id]
    filmes = []
    for _, assistiu in lista_assistiu.iterrows():
        filmes.append({
            "id": assistiu["filme"],
            "avaliacao": assistiu["rating"],
            "data": assistiu["data"]
        })
    usuarios.append({
        "media": float(lista_assistiu["rating"].mean()),
        "user_similarity": [],
        "id": int(cust_id),
        "filmes": filmes
    })

print ("[%s] 100%%   " % ('#'*100))
print ("Salvando no arquivo json")
with open('data.json', 'w', encoding='utf8') as outfile:
    outfile.write(json.dumps(usuarios,
                                indent=4,
                                separators=(',', ': ')))

In [143]:
# def teste_v():
#     index = list(data_set[data_set.rating.isna()].index)
# #     print (index)
#     df = data_set
#     df["filme"] = '-1'

#     for i in range(0, len(index) - 1):
#         df.iloc[index[i] + 1:index[i + 1], 3] = df.cust_id[index[i]].split(":")[0]

#     df.iloc[index[-1] + 1:len(df), 3] = df.cust_id[index[-1]].split(":")[0]
#     # # df = data_set
#     # # df["filme"] = 2
#     # # df
#     # df.iloc[index[0] + 1:index[1], 2] = 12
#     # df.iloc[first + 1:index[i], 2] = df.Cust_Id[first].split(":")[0]
#     df = df[~df.rating.isna()]
#     print (len(df))
    
# %time teste_v()
# df
# df.Cust_Id[0].split(":")[0]

index = list(data_set.query("rating != rating").index)
print ("Filmes" , len(index))
print ("Pegou os ids dos filmes")
data_set.loc[index, "cust_id"] = filmes

499852
CPU times: user 2.06 s, sys: 6.61 ms, total: 2.06 s
Wall time: 2.08 s


In [6]:
usuarios = []
users = df.cust_id.unique()
print (len(users))
for cust_id in users:
#     print (cust_id)
    lista_assistiu = df.query('cust_id == @cust_id')
#     print (lista_assistiu)
    filmes = []
#     print(cust_id, "-->", lista_assistiu.rating.mean())
    for _, assistiu in lista_assistiu.iterrows():
#         print (assistiu)
        filmes.append({
            "id": assistiu["filme"],
            "avaliacao": assistiu["rating"],
            "data": assistiu["data"]
        })
    usuarios.append({
        "media": float(lista_assistiu["rating"].mean()),
        "user_similarity": [],
        "id": int(cust_id),
        "filmes": filmes
    })
    
with open('data.json', 'w', encoding='utf8') as outfile:
    outfile.write(json.dumps(usuarios,
                                      indent=4,
                                      separators=(',', ': ')))
# print (usuarios)
# df[df.cust_id == '2442']
# for i in df.cust_id[:20]:
#     print (i)

In [78]:
lista_assistiu = df[df.cust_id == '2442']
#     print (lista_assistiu)
filmes = []
print(cust_id, "-->", lista_assistiu.rating.mean())
for _, assistiu in lista_assistiu.iterrows():
#         print (assistiu)
    filmes.append({
        "id": assistiu.filme,
        "avaliacao": assistiu.rating,
        "data": assistiu["data"]
    })
    print (filmes)

141045 --> 3.0
[{'id': '1', 'avaliacao': 3.0, 'data': '2004-04-14'}]


In [12]:
with open('data.json', 'w', encoding='utf8') as outfile:
    outfile.write(json.dumps([{"teste": "123", "ert": ["tresd"]}],
                                          indent=4,
                                          separators=(',', ': ')))

In [4]:
gdf = pd.read_csv('./combined_data_1.csv', nrows=5000)
gdf.head(5)


,cust_id,rating,year,month,day
0,1,NaN,NaN,NaN,NaN
1,1488844,3.0,2005.0,9.0,6.0
2,822109,5.0,2005.0,5.0,13.0
3,885013,4.0,2005.0,10.0,19.0
4,30878,4.0,2005.0,12.0,26.0


In [41]:
# def teste_v():
#     index = list(data_set[data_set.rating.isna()].index)
# #     print (index)
#     df = data_set
#     df["filme"] = '-1'

#     for i in range(0, len(index) - 1):
#         df.iloc[index[i] + 1:index[i + 1], 3] = df.cust_id[index[i]].split(":")[0]

#     df.iloc[index[-1] + 1:len(df), 3] = df.cust_id[index[-1]].split(":")[0]
#     # # df = data_set
#     # # df["filme"] = 2
#     # # df
#     # df.iloc[index[0] + 1:index[1], 2] = 12
#     # df.iloc[first + 1:index[i], 2] = df.Cust_Id[first].split(":")[0]
#     df = df[~df.rating.isna()]
#     print (len(df))
def df():
    indexes = list(df_gdf.query("rating != rating").index)
    def apply_filme(cust_id, filme):
        tam = len(cust_id)
#         print (tam, type(filme))
#         for i in range(0, len(index)):
#             filme[index[i]:index[i+1]] = cust_id[i]
    df_gdf.apply_rows(apply_filme, incols=["cust_id"], outcols={})
    print (df_gdf.head(5).to_pandas())
#     print (indexes)

%time df()
    

CudaAPIError: [999] Call to cuMemAlloc results in CUDA_ERROR_UNKNOWN

In [30]:
def ds():
    indexes = list(gdf.query("rating != rating").index)
#     print (indexes)
    
%time ds()

CPU times: user 4.7 ms, sys: 223 µs, total: 4.92 ms
Wall time: 7.15 ms
